In [1]:
import pandas as pd
import requests
import json

import sqlite3
con = sqlite3.connect ('database.sqlite')
cur = con.cursor()

In [2]:
cur.execute("""select * From Matches;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [48]:
cur.execute("""SELECT *
               FROM Matches
               WHERE Season == '2011'
               AND ((Div = 'D1' OR 'D2'))
               ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
...,...,...,...,...,...,...,...,...,...
301,1648,D1,2011,2011-08-06,Hannover,Hoffenheim,2,1,H
302,1649,D1,2011,2011-12-10,Nurnberg,Hoffenheim,0,2,A
303,1650,D1,2011,2011-08-20,Augsburg,Hoffenheim,0,2,A
304,1651,D1,2011,2012-04-15,Freiburg,Hoffenheim,0,0,D


In [49]:
df.shape

(306, 9)

# German Premier League Only
    

In [14]:
df = df[df.Div != 'E0']
df.shape

(612, 9)

In [43]:
df['Div'].unique()

array(['D1', 'D2'], dtype=object)

In [15]:
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


# Home goals in totals


In [45]:
cur.execute("""SELECT Match_ID , Date , Season, HomeTeam , Div , SUM(FTHG) AS Home_Goals
              FROM Matches
              WHERE Season == '2011'
              GROUP BY Hometeam
              AND ((Div ='D1' OR Div = 'D2'))
              ;""")
dfH = pd.DataFrame(cur.fetchall())
dfH.columns = [x[0] for x in cur.description]
dfH.head()

,Match_ID,Date,Season,HomeTeam,Div,Home_Goals
0,1092,2012-03-31,2011,Nurnberg,D1,1600


In [35]:
cur.execute("""SELECT Match_ID , Date , AwayTeam AS Team, Season , Div , SUM(FTAG) AS A_Goals
              FROM Matches
              WHERE Season == '2011'
              GROUP BY 1;""")
dfA = pd.DataFrame(cur.fetchall())
dfA.columns = [x[0] for x in cur.description]
dfA.head()

,Match_ID,Date,Team,Season,Div,A_Goals
0,1092,2012-03-31,Bayern Munich,2011,D1,1
1,1093,2011-12-11,Bayern Munich,2011,D1,2
2,1094,2011-08-13,Bayern Munich,2011,D1,1
3,1095,2011-11-27,Bayern Munich,2011,D1,2
4,1096,2012-02-18,Bayern Munich,2011,D1,0


In [36]:
new_df = pd.merge(dfH,dfA, on=['Team'])

In [37]:
new_df.head()

,Match_ID_x,Date_x,Team,Season_x,Div_x,H_Goals,Match_ID_y,Date_y,Season_y,Div_y,A_Goals
0,1092,2012-03-31,Nurnberg,2011,D1,0,1279,2012-04-14,2011,D1,2
1,1092,2012-03-31,Nurnberg,2011,D1,0,1280,2012-01-27,2011,D1,0
2,1092,2012-03-31,Nurnberg,2011,D1,0,1281,2012-03-10,2011,D1,1
3,1092,2012-03-31,Nurnberg,2011,D1,0,1282,2012-02-25,2011,D1,1
4,1092,2012-03-31,Nurnberg,2011,D1,0,1283,2012-04-28,2011,D1,3


In [38]:
#Delete extra columns

In [40]:
new_df.drop('Season_y', axis = 1 , inplace = True)

In [41]:
new_df.drop('Div_y', axis = 1 , inplace = True)

In [42]:
new_df.head()

,Match_ID_x,Date_x,Team,Season_x,Div_x,H_Goals,Match_ID_y,Date_y,A_Goals
0,1092,2012-03-31,Nurnberg,2011,D1,0,1279,2012-04-14,2
1,1092,2012-03-31,Nurnberg,2011,D1,0,1280,2012-01-27,0
2,1092,2012-03-31,Nurnberg,2011,D1,0,1281,2012-03-10,1
3,1092,2012-03-31,Nurnberg,2011,D1,0,1282,2012-02-25,1
4,1092,2012-03-31,Nurnberg,2011,D1,0,1283,2012-04-28,3
